In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score


In [4]:
df = pd.read_csv('E:\\CODE\\python\\Machine_learning\\Do_an_cuoiky\\datasets\\Heart_Attack\\Heart Attack.csv')
# df = pd.read_csv('E:\\CODE\\python\\Machine_learning\\decision_tree\\heart_disease\\heart\\heart.csv')
df.head()

,age,gender,impluse,pressurehight,pressurelow,glucose,kcm,troponin,class
0,64,1,66,160,83,160.0,1.80,0.012,negative
1,21,1,94,98,46,296.0,6.75,1.060,positive
2,55,1,64,160,77,270.0,1.99,0.003,negative
3,64,1,70,120,55,270.0,13.87,0.122,positive
4,55,1,64,112,65,300.0,1.08,0.003,negative


In [5]:
df.shape

(1319, 9)

In [6]:
from sklearn.preprocessing import LabelEncoder
 
# Creating a instance of label Encoder.
le = LabelEncoder()
 
# Using .fit_transform function to fit label
# encoder and return encoded label
label = le.fit_transform(df['class'])
 
label

array([0, 1, 0, ..., 1, 1, 1])

In [7]:
# removing the column 'class' from df
# as it is of no use now.
df.drop("class", axis=1, inplace=True)
 
# Appending the array to our dataFrame 
# with column name 'class'
df["class"] = label
 
# printing Dataframe
df

,age,gender,impluse,pressurehight,pressurelow,glucose,kcm,troponin,class
0,64,1,66,160,83,160.0,1.80,0.012,0
1,21,1,94,98,46,296.0,6.75,1.060,1
2,55,1,64,160,77,270.0,1.99,0.003,0
3,64,1,70,120,55,270.0,13.87,0.122,1
4,55,1,64,112,65,300.0,1.08,0.003,0
...,...,...,...,...,...,...,...,...,...
1314,44,1,94,122,67,204.0,1.63,0.006,0
1315,66,1,84,125,55,149.0,1.33,0.172,1
1316,45,1,85,168,104,96.0,1.24,4.250,1
1317,54,1,58,117,68,443.0,5.80,0.359,1


In [8]:
X = df.drop(columns = ['class']).values
y = df['class'].values

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from imblearn.over_sampling import RandomOverSampler
X_train, y_train = RandomOverSampler().fit_resample(X_train, y_train)

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
X.shape

(1319, 8)

In [13]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 4)
X = pca.fit_transform(X)
X.shape


(1319, 4)

In [14]:
from sklearn.decomposition import PCA

pca = PCA()
X_pca = pca.fit_transform(X)
explained_variance_ratio = pca.explained_variance_ratio_

cumulative_variance = 0
n_components_to_retain = 0

for i, ratio in enumerate(explained_variance_ratio):
    cumulative_variance += ratio
    if cumulative_variance >= 0.95:  # Hoặc sử dụng giá trị mong muốn của bạn
        n_components_to_retain = i + 1
        break

print("Number of components to retain:", n_components_to_retain)


Number of components to retain: 4


In [15]:
sum(y_train == 0)

647

In [16]:
sum(y_train == 1)


647

## SVM

- Optimization và Grid Search:


In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],
              'C': [0.1, 1, 10, 100]}

grid_search = GridSearchCV(SVC(), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(best_params)
print(best_model)

{'C': 100, 'kernel': 'linear'}
SVC(C=100, kernel='linear')


In [18]:
from sklearn import svm
df = svm.SVC(kernel='linear', C=100)
df.fit(X_train, y_train)
y_pred = df.predict(X_test)

In [19]:
from sklearn import metrics
print("confusion_matrix: \n", metrics.confusion_matrix(y_test, y_pred))
print("accuracy_score: ", metrics.accuracy_score(y_test, y_pred))
print("f1_score: ", metrics.f1_score(y_test, y_pred))
print("recall_score: ", metrics.recall_score(y_test, y_pred))
print("precision_score: ", metrics.precision_score(y_test, y_pred))

confusion_matrix: 
 [[ 95   6]
 [ 14 149]]
accuracy_score:  0.9242424242424242
f1_score:  0.9371069182389937
recall_score:  0.9141104294478528
precision_score:  0.9612903225806452


## MLP

In [20]:
from sklearn.neural_network import MLPClassifier

# Thiết lập lưới tìm kiếm
param_grid_mlp = {'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
                  'alpha': [0.001, 0.01, 0.1]}

# Tạo mô hình MLP
mlp = MLPClassifier(max_iter=500)

# Tìm kiếm trên lưới
grid_search_mlp = GridSearchCV(mlp, param_grid_mlp, cv=5, scoring='accuracy')
grid_search_mlp.fit(X_train, y_train)

# In kết quả
print("Best parameters for MLP:", grid_search_mlp.best_params_)

# Đánh giá mô hình trên tập kiểm thử
y_pred_mlp = grid_search_mlp.predict(X_test)
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
print("Accuracy on test set:", accuracy_mlp)

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the opti

Best parameters for MLP: {'alpha': 0.1, 'hidden_layer_sizes': (50, 50, 50)}
Accuracy on test set: 0.8295454545454546


In [29]:
mlp = MLPClassifier(hidden_layer_sizes=(50, 50, 50), alpha=0.1)
# mlp = MLPClassifier()
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_test)

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [30]:
from sklearn import metrics
print("confusion_matrix: \n", metrics.confusion_matrix(y_test, y_pred_mlp))
print("accuracy_score: ", metrics.accuracy_score(y_test, y_pred_mlp))
print("f1_score: ", metrics.f1_score(y_test, y_pred_mlp))
print("recall_score: ", metrics.recall_score(y_test, y_pred_mlp))
print("precision_score: ", metrics.precision_score(y_test, y_pred_mlp))


confusion_matrix: 
 [[ 70  31]
 [ 10 153]]
accuracy_score:  0.8446969696969697
f1_score:  0.8818443804034581
recall_score:  0.9386503067484663
precision_score:  0.8315217391304348


## Logistic_regression

- Optimization và Grid Search:

In [23]:
from sklearn.linear_model import LogisticRegression

# Thiết lập lưới tìm kiếm
param_grid_lr = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

# Tạo mô hình Logistic Regression
logistic_regression = LogisticRegression()

# Tìm kiếm trên lưới
grid_search_lr = GridSearchCV(logistic_regression, param_grid_lr, cv=5, scoring='accuracy')
grid_search_lr.fit(X_train, y_train)

# In kết quả
print("Best parameters for Logistic Regression:", grid_search_lr.best_params_)

# Đánh giá mô hình trên tập kiểm thử
y_pred_lr = grid_search_lr.predict(X_test)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print("Accuracy on test set:", accuracy_lr)

Best parameters for Logistic Regression: {'C': 100}
Accuracy on test set: 0.8901515151515151


In [24]:
from sklearn.linear_model import LogisticRegression
lgr = LogisticRegression(C=100).fit(X_train, y_train)
y_pred_lgr = lgr.predict(X_test)

In [25]:
from sklearn import metrics
print("confusion_matrix: \n", metrics.confusion_matrix(y_test, y_pred_lgr))
print("accuracy_score: ", metrics.accuracy_score(y_test, y_pred_lgr))
print("f1_score: ", metrics.f1_score(y_test, y_pred_lgr))
print("recall_score: ", metrics.recall_score(y_test, y_pred_lgr))
print("precision_score: ", metrics.precision_score(y_test, y_pred_lgr))


confusion_matrix: 
 [[ 94   7]
 [ 22 141]]
accuracy_score:  0.8901515151515151
f1_score:  0.9067524115755626
recall_score:  0.8650306748466258
precision_score:  0.9527027027027027


## Decision Tree

In [26]:
from sklearn.tree import DecisionTreeClassifier
# Thiết lập lưới tìm kiếm
param_grid_dt = {'max_depth': [3, 4, 5, 6, 7, None],
                 'min_samples_split': [2, 5, 10],
                 'min_samples_leaf': [1, 2, 4],
                 'ccp_alpha': [0.001, 0.01, 0.1, 1.0],
                 'criterion': ['entropy', 'gini']}

# Tạo mô hình Decision Tree
dt = DecisionTreeClassifier()

# Tìm kiếm trên lưới
grid_search_dt = GridSearchCV(dt, param_grid_dt, cv=5, scoring='accuracy')
grid_search_dt.fit(X_train, y_train)

# In kết quả
print("Best parameters for Decision Tree:", grid_search_dt.best_params_)

# Đánh giá mô hình trên tập kiểm thử
y_pred_dt = grid_search_dt.predict(X_test)
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Accuracy on test set:", accuracy_dt)

Best parameters for Decision Tree: {'ccp_alpha': 0.001, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 2}
Accuracy on test set: 0.9772727272727273


In [27]:
dt = DecisionTreeClassifier(ccp_alpha=0.001, max_depth = 5, min_samples_leaf=1, min_samples_split=2, criterion='entropy').fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

In [28]:
from sklearn import metrics
print("confusion_matrix: \n", metrics.confusion_matrix(y_test, y_pred_dt))
print("accuracy_score: ", metrics.accuracy_score(y_test, y_pred_dt))
print("f1_score: ", metrics.f1_score(y_test, y_pred_dt))
print("recall_score: ", metrics.recall_score(y_test, y_pred_dt))
print("precision_score: ", metrics.precision_score(y_test, y_pred_dt))


confusion_matrix: 
 [[ 98   3]
 [  3 160]]
accuracy_score:  0.9772727272727273
f1_score:  0.98159509202454
recall_score:  0.9815950920245399
precision_score:  0.9815950920245399
